VGG-19 IMPLEMENTATION

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
# from google.colab import drive
import random

In [ ]:
# drive.mount('/content/drive')

In [ ]:
IMG_SIZE = 224  # 224x224 as mentioned in paper
NUM_SAMPLES = 500

In [ ]:
dataset_path = 'E://data'
train_path = os.path.join(dataset_path, 'train')
test_path = os.path.join(dataset_path, 'test')

In [ ]:
train_benign_path = os.path.join(train_path, 'benign')
train_malignant_path = os.path.join(train_path, 'malignant')
test_benign_path = os.path.join(test_path, 'benign')
test_malignant_path = os.path.join(test_path, 'malignant')

In [ ]:
print("Checking dataset structure...")
print(f"Dataset folder exists: {os.path.exists(dataset_path)}")
print(f"Train folder exists: {os.path.exists(train_path)}")
print(f"Test folder exists: {os.path.exists(test_path)}")

In [ ]:
print("\nTrain folders:")
print(f"  Train/Benign exists: {os.path.exists(train_benign_path)}")
print(f"  Train/Malignant exists: {os.path.exists(train_malignant_path)}")

print("\nTest folders:")
print(f"  Test/Benign exists: {os.path.exists(test_benign_path)}")
print(f"  Test/Malignant exists: {os.path.exists(test_malignant_path)}")

In [ ]:
train_benign_count = len([f for f in os.listdir(train_benign_path) if f.endswith('.jpg')])
print(f"  Train/Benign images: {train_benign_count}")

train_malignant_count = len([f for f in os.listdir(train_malignant_path) if f.endswith('.jpg')])
print(f"  Train/Malignant images: {train_malignant_count}")

test_benign_count = len([f for f in os.listdir(test_benign_path) if f.endswith('.jpg')])
print(f"  Test/Benign images: {test_benign_count}")

test_malignant_count = len([f for f in os.listdir(test_malignant_path) if f.endswith('.jpg')])
print(f"  Test/Malignant images: {test_malignant_count}")

Load Images

In [ ]:
def load_images(folder_path, label, max_samples=500):
    images = []
    labels = []

    # Get all jpg files
    files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]

    # Randomly select max_samples files
    if len(files) > max_samples:
        files = random.sample(files, max_samples)

    print(f"Loading {len(files)} images from {folder_path.split('/')[-1]} folder...")

    # Define transforms
    transform = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),  # Converts PIL image to tensor and normalizes to [0,1]
    ])

    for i, filename in enumerate(files):
        try:
            # Load and transform image
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path).convert('RGB')
            img_tensor = transform(img)

            # Convert to numpy for consistency with rest of code
            img_array = img_tensor.permute(1, 2, 0).numpy()  # CHW -> HWC

            images.append(img_array)
            labels.append(label)

            if (i + 1) % 100 == 0:
                print(f"  Processed {i + 1}/{len(files)} images")

        except Exception as e:
            print(f"  Error loading {filename}: {e}")
            continue

    return np.array(images), np.array(labels)

In [ ]:
train_benign_images, train_benign_labels = load_images(train_benign_path, 0, NUM_SAMPLES)  # 0 for benign
train_malignant_images, train_malignant_labels = load_images(train_malignant_path, 1, NUM_SAMPLES)  # 1 for malignant

In [ ]:
# Combine training data
X_train = np.concatenate([train_benign_images, train_malignant_images], axis=0)
y_train = np.concatenate([train_benign_labels, train_malignant_labels], axis=0)

In [ ]:
test_benign_images, test_benign_labels = load_images(test_benign_path, 0, NUM_SAMPLES)  # 0 for benign
test_malignant_images, test_malignant_labels = load_images(test_malignant_path, 1, NUM_SAMPLES)  # 1 for malignant

In [ ]:
# Combine test data
X_test = np.concatenate([test_benign_images, test_malignant_images], axis=0)
y_test = np.concatenate([test_benign_labels, test_malignant_labels], axis=0)

In [ ]:
print(f"\n=== Dataset Summary ===")
print(f"Training set:")
print(f"  Total images: {len(X_train)}")
print(f"  Benign images: {np.sum(y_train == 0)}")
print(f"  Malignant images: {np.sum(y_train == 1)}")

In [ ]:
print(f"\nTest set:")
print(f"  Total images: {len(X_test)}")
print(f"  Benign images: {np.sum(y_test == 0)}")
print(f"  Malignant images: {np.sum(y_test == 1)}")

In [ ]:
print(f"\nImage specifications:")
print(f"  Image shape: {X_train[0].shape}")
print(f"  Pixel value range: [{X_train.min():.3f}, {X_train.max():.3f}]")

In [ ]:
# Step 10: Display sample images
def display_sample_images(X, y, title="Sample Images", num_samples=6):
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    axes = axes.ravel()

    for i in range(num_samples):
        idx = random.randint(0, len(X) - 1)
        img = X[idx]
        label = 'Malignant' if y[idx] == 1 else 'Benign'

        axes[i].imshow(img)
        axes[i].set_title(f'{label}')
        axes[i].axis('off')

    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

In [ ]:
# Display sample images from training set
print("\n=== Displaying Sample Training Images ===")
display_sample_images(X_train, y_train, "Training Set - Sample Images")

print("\nDataset loading completed successfully!")
print("\nVariables created:")
print("- X_train: Training images")
print("- X_test: Test images")
print("- y_train: Training labels (0=benign, 1=malignant)")
print("- y_test: Test labels (0=benign, 1=malignant)")

Dataset Preparation

In [ ]:
# Import additional libraries
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import torchvision.transforms as transforms
import numpy as np

In [ ]:
# Step 1: One-hot Encoding
print("1. Converting labels to one-hot encoding...")

# Create label encoder
label_encoder = LabelEncoder()

# Fit and transform training labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert to one-hot encoding using PyTorch
y_train_onehot = F.one_hot(torch.tensor(y_train_encoded), num_classes=2).float().numpy()
y_test_onehot = F.one_hot(torch.tensor(y_test_encoded), num_classes=2).float().numpy()

print(f"Original labels shape: {y_train.shape}")
print(f"One-hot encoded labels shape: {y_train_onehot.shape}")
print(f"Label mapping: {dict(enumerate(label_encoder.classes_))}")

In [ ]:

# Show examples
print(f"Example - Original: {y_train[:5]} -> One-hot: {y_train_onehot[:5]}")

In [ ]:
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train, y_train_onehot,
    test_size=0.3,           # 30% for validation
    random_state=42,
    stratify=y_train_encoded  # Stratified sampling to maintain class balance
)

In [ ]:
print(f"Final training set: {len(X_train_final)} images")
print(f"Validation set: {len(X_val)} images")
print(f"Test set: {len(X_test)} images")

In [ ]:
# Check class distribution
print(f"\nClass distribution in final training set:")
print(f"  Benign: {np.sum(np.argmax(y_train_final, axis=1) == 0)}")
print(f"  Malignant: {np.sum(np.argmax(y_train_final, axis=1) == 1)}")

In [ ]:
print(f"\nClass distribution in validation set:")
print(f"  Benign: {np.sum(np.argmax(y_val, axis=1) == 0)}")
print(f"  Malignant: {np.sum(np.argmax(y_val, axis=1) == 1)}")

In [ ]:
# Data Augmentation using PyTorch transforms
print("   Setting up data augmentation...")
print("   Rotation range: 15 degrees (as mentioned in paper)")

# ImageNet normalization values for VGG19
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

# Create data augmentation transforms for training
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(15),          # Rotate images by up to 15 degrees
    transforms.RandomHorizontalFlip(p=0.5), # Flip images horizontally
    transforms.ColorJitter(brightness=0.1, contrast=0.1),  # Color jittering
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), shear=0.1),  # Translation and shear
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),  # VGG19 ImageNet normalization
])

# Validation and test transforms (no augmentation)
val_test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),  # VGG19 ImageNet normalization
])

print("Data augmentation parameters:")
print(f"  - Rotation range: 15°")
print(f"  - Horizontal flip: 50% probability")
print(f"  - Color jitter: brightness/contrast ±10%")
print(f"  - Translation: ±10%")
print(f"  - Shear range: 10%")
print(f"  - ImageNet normalization: mean={IMAGENET_MEAN}, std={IMAGENET_STD}")

In [ ]:
# Validation and test data generators (no augmentation, only rescaling)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
val_datagen = ImageDataGenerator(rescale=1.0)
test_datagen = ImageDataGenerator(rescale=1.0)

print("Data augmentation parameters:")
print(f"  - Rotation range: 15°")
print(f"  - Width/Height shift: 10%")
print(f"  - Horizontal flip: Yes")
print(f"  - Zoom range: 10%")
print(f"  - Shear range: 10%")

In [ ]:
BATCH_SIZE = 32

In [ ]:
# Create PyTorch datasets and dataloaders
from torch.utils.data import Dataset, DataLoader

class SkinCancerDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        # Convert numpy array to uint8 for PIL transforms
        if image.max() <= 1.0:
            image = (image * 255).astype(np.uint8)

        if self.transform:
            image = self.transform(image)
        else:
            image = torch.tensor(image).permute(2, 0, 1).float() / 255.0  # HWC -> CHW and normalize

        # Convert one-hot label to class index for PyTorch
        if len(label.shape) > 0 and label.shape[0] > 1:  # one-hot encoded
            label = torch.tensor(np.argmax(label)).long()
        else:
            label = torch.tensor(label).long()

        return image, label

# Create datasets
train_dataset = SkinCancerDataset(X_train_final, y_train_final, transform=train_transform)
val_dataset = SkinCancerDataset(X_val, y_val, transform=val_test_transform)
test_dataset = SkinCancerDataset(X_test, y_test_onehot, transform=val_test_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
print(f"   Data loaders created with batch size: {BATCH_SIZE}")
print(f"   Training batches per epoch: {len(train_loader)}")
print(f"   Validation batches per epoch: {len(val_loader)}")
print(f"   Test batches: {len(test_loader)}")

In [ ]:
# Step 5: Visualize augmented images
import matplotlib.pyplot as plt

def show_augmented_images(dataloader, num_images=6):
    """Display original and augmented images"""
    # Get a batch from the dataloader
    dataiter = iter(dataloader)
    batch_images, batch_labels = next(dataiter)

    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()

    for i in range(min(num_images, len(batch_images))):
        # Convert tensor to numpy for display (CHW -> HWC)
        img = batch_images[i].permute(1, 2, 0).numpy()
        # Ensure values are in [0,1] range for display
        img = np.clip(img, 0, 1)

        label = batch_labels[i].item()
        label_name = 'Malignant' if label == 1 else 'Benign'

        axes[i].imshow(img)
        axes[i].set_title(f'Augmented {label_name}')
        axes[i].axis('off')

    plt.suptitle('Sample Augmented Training Images', fontsize=16)
    plt.tight_layout()
    plt.show()

print("   Displaying sample augmented images...")
show_augmented_images(train_loader)

In [ ]:

# Reset the dataloader

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)


In [ ]:
# Step 6: Summary
print("\n=== Dataset Preparation Summary ===")
print("✓ Labels converted to one-hot encoding")
print("✓ Data split using stratified sampling (70% train, 30% validation)")
print("✓ Data augmentation applied to training set")
print("✓ Data generators created for training, validation, and testing")
print("\nDataset is ready for VGG-19 model training!")

print("\nVariables available for next step:")
print("- train_generator: Training data with augmentation")
print("- val_generator: Validation data")
print("- test_generator: Test data")
print("- X_train_final, y_train_final: Final training arrays")
print("- X_val, y_val: Validation arrays")
print("- X_test, y_test_onehot: Test arrays with one-hot labels")